In [ ]:
from datetime import datetime, timedelta
import csv
import sqlite3
import requests
from bs4 import BeautifulSoup
import json
from typing import Dict, List
import re

In [ ]:
# Create or connect to the database
conn = sqlite3.connect("Inventory_mgnt.db")
conn.execute("PRAGMA foreign_keys = ON")  # Enforces foreign key constraints - execute function id=s needed to execute queries...
cursor = conn.cursor()  # this executes the query

In [ ]:


BASE_URL = "https://books.toscrape.com/catalogue/page-{}.html"

def scrape_books(pages=25):
    books = []

    for page in range(1, pages + 1):
        url = BASE_URL.format(page)
        print(f"Scraping page {page}...")

        response = requests.get(url)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, "html.parser")
        articles = soup.find_all("article", class_="product_pod")

        for article in articles:
            name = article.h3.a["title"]

            price_text = article.find("p", class_="price_color").text
            price = float(re.sub(r"[^\d.]", "", price_text))


            books.append({
                "name": name,
                "price": price,
                #"source": f"Page {page}"
            })

    return books

In [ ]:
# scrap all the pages
books = scrape_books(25)

print(f"\nTotal books scraped: {len(books)}")

for book in books[:2]:
    print(book)

In [ ]:
filename = "inventory_sample.csv"

with open(filename, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.DictWriter(file, fieldnames=books[0].keys())
    writer.writeheader()
    writer.writerows(books)

print("Data saved to inventory_sample.csv")

In [ ]:
# Create a Product table
cursor.execute("""     
CREATE TABLE IF NOT EXISTS Product (
    product_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    price FLOAT NOT NULL,
    supplier VARCHAR DEFAULT 'Generic'
               
)
""")
conn.commit() #saves any changes made to the database permanently. 

In [ ]:
# Read CSV and Insert Into Tables

def insert_from_csv(csv_file):
    conn = sqlite3.connect("Inventory_mgnt.db")
    cursor = conn.cursor()
    
    with open(csv_file, newline="", encoding="utf-8") as file:
        reader = csv.DictReader(file)

        for row in reader:
            name = row["name"]
            price = float(row["price"])
            
            # Insert into product
            cursor.execute("""
                INSERT INTO product (name, price)
                VALUES (?, ?)
            """, (name, price))

            product_id = cursor.lastrowid

            # Insert into inventory
            #cursor.execute("""
            #    INSERT INTO inventory (product_id, stock, reorder_level, units_sold)
            #    VALUES (?, ?, ?, 0)
            #""", (product_id, stock, reorder_level))

    conn.commit()
    conn.close()

insert_from_csv("inventory_sample.csv")
print("Data inserted into table successfully")

In [ ]:
cursor.execute("""     
CREATE TABLE IF NOT EXISTS inventory (
    inventory_id    INTEGER PRIMARY KEY AUTOINCREMENT,           
    product_id      INTEGER NOT NULL,
    stock           INTEGER NOT NULL CHECK (stock >= 0),
    reorder_level   INTEGER NOT NULL CHECK (reorder_level >= 0),
    units_sold      INTEGER NOT NULL DEFAULT 0,
    FOREIGN KEY (product_id) REFERENCES product(product_id) ON DELETE CASCADE
)
""")
conn.commit()

In [ ]:
class Product:
    def __init__(self, product_id, name, price, stock, supplier, reorder_level):
        self.product_id = product_id
        self.name = name
        self.price = price
        self.stock = stock
        self.supplier = supplier
        self.reorder_level = reorder_level
        self.units_sold = 0

    def sell(self, quantity):
        if quantity > self.stock:
            print(f"Not enough stock for {self.name}")
            return False

        self.stock -= quantity
        self.units_sold += quantity
        return True

    def needs_restock(self):
        return self.stock <= self.reorder_level

    def __str__(self):
        return f"{self.name} | Stock: {self.stock} | Sold: {self.units_sold}"

    
    


In [ ]:
class Inventory:
    def __init__(self):
        self.products = {}

    def add_product(self, product):
        self.products[product.product_id] = product
        print(f"Added {product.name}")

    def sell_product(self, product_id, quantity):
        product = self.products.get(product_id)

        if not product:
            print(" Product not found")
            return

        if product.sell(quantity):
            print(f" Sold {quantity} of {product.name}")
            if product.needs_restock():
                print(f" Low stock alert for {product.name}")

    def low_stock_alerts(self):
        print("\n LOW STOCK ALERTS")
        for product in self.products.values():
            if product.needs_restock():
                print(f"{product.name} (Stock: {product.stock})")

